In [ ]:
# dependencies import
from common_dependencies import *
import model_versions as mv
logger = logging.getLogger(f'main.model_train')
# детерминация случайных величин, отвечающих за выбор первоначальных весов и биасов

In [ ]:
# train params that can be easily changed
tf.compat.v1.set_random_seed(290)
tf.random.set_seed(290)
np.random.seed(290)
CROP_SIZE = 16
CROP_STEP = 1
MAX_VAL = 1000
XSHIFT = 200
dataset_desc = {'train': (DatasetPartDescription(PATH_TO_DATA['run_1'],DataCrop(120,0,240,MAX_VAL),SlidingCrop(CROP_SIZE,CROP_STEP),XSHIFT)
                          
                         ,),
                'val': (DatasetPartDescription(PATH_TO_DATA['run_1'],DataCrop(0,0,120,MAX_VAL),SlidingCrop(CROP_SIZE,CROP_STEP),XSHIFT),
                        DatasetPartDescription(PATH_TO_DATA['run_1'],DataCrop(360,0,MAX_VAL,MAX_VAL),SlidingCrop(CROP_SIZE,CROP_STEP),XSHIFT),
                        DatasetPartDescription(PATH_TO_DATA['run_2'],DataCrop(0,0,MAX_VAL,40),SlidingCrop(CROP_SIZE,CROP_STEP),XSHIFT)
                        
                        ,)
               }

In [ ]:
#входные и выходные данные
# reading
dataset = {'train':dict(zip(['x','y','bin'], [np.array(list(gen)) for gen in chain_dataset_gens(dataset_desc['train'])])),
           'val':dict(zip(['x','y','bin'], [np.array(list(gen)) for gen in chain_dataset_gens(dataset_desc['val'])]))}

# displaying
logger.debug('Dataset')
for dataset_part_name, dataset_part in dataset.items():
    logger.debug('|'*8+dataset_part_name+'|'*8)
    for data_part_name, data_part in dataset_part.items():
        logger.debug(f'{data_part_name}.shape: {data_part.shape}')

In [ ]:
#выборка данных
# show parts took for learning
all_rects = {'run_1': {'train':None,'val':None}, 
             'run_2': {'train':None,'val':None}}
rects_colors = {'train':'red', 'val':'green'}

for run_name in all_rects.keys():
    x_df, y_df = dw.get_x_and_y_data(*PATH_TO_DATA[run_name])
    x_df = None
    y_df = dw.roll_df(y_df, XSHIFT, 1)
    for dataset_part_name in all_rects[run_name].keys():
        # get all DatasetPartDescription for train, val or test
        dataset_part_desc = dataset_desc[dataset_part_name]
        # get all DatasetPartDescription for current run_name (run_1 or run_2)
        dataset_part_desc = [dataset_part for dataset_part in dataset_part_desc if re.findall(r'run_\d', dataset_part.data_path_tuple[0])[0] == run_name]
        # put rects list to all_rects[run_name][dataset_part_name]
        all_rects[run_name][dataset_part_name] = [Rectangle((dataset_part.file_data_crop.left, dataset_part.file_data_crop.top), 
                           dataset_part.file_data_crop.width, dataset_part.file_data_crop.height, 
                           facecolor=rects_colors[dataset_part_name], alpha=0.5) for dataset_part in dataset_part_desc]
    res_rects = list(itertools.chain(*[run_rects for run_rects_name, run_rects in all_rects[run_name].items()]))
    if res_rects:
        dw.draw_defects_map_with_rectangles_owerlap(y_df, res_rects, title = f'The parts took for learning from run_1 (red - train, green - validate, other - test)')

In [ ]:
#### всякие константы для последующей работы

#///////////////////////////////// для компиляции 

CMP_learning_rate = 0.005 #0.0000002 # шаг сходимости back propogation
#CMP_solver = keras.optimizers.Adam(CMP_learning_rate) # оптимизатор
CMP_solver = keras.optimizers.SGD(CMP_learning_rate) # оптимизатор
CMP_loss_funcs = keras.losses.BinaryCrossentropy() #BinaryCrossentropy() 
CMP_metrics = [keras.metrics.BinaryAccuracy(name='BinaryAccuracy'),
               keras.metrics.MeanSquaredError(name='MeanSquaredError'),
               keras.metrics.TruePositives(name='TruePositives'),
               keras.metrics.FalsePositives(name='FalsePositives'),
               keras.metrics.TrueNegatives(name='TrueNegatives'),
               keras.metrics.FalseNegatives(name='FalseNegatives'),     
               keras.metrics.Precision(name='Precision'),
               keras.metrics.Recall(name='Recall'),
               keras.metrics.AUC(name='AUC')]
#///////////////////////////////// для колбэков

# для Early_stopping
ES_patience = 3 # кол-во эпох без улучшений
ES_min_delta = 0.00001 # минимальное улучшение параметра за cur_patience
ES_monitor_parametr =  'loss' # отслеживаемый параметр 
ES_save_best_weights = True # сохранять ли веса нейронки с лучшими результатами
Es_mode = 'max'

   # для ReduceLROnPlateau
RLPOP_monitor_parametr = 'loss'  # отслеживаемый параметр 
RLPOP_factor = 0.3 # множитель для расчета нового шага сходимости (new_learning_rate = old_learning_rate*RLPOP_factor)
RLPOP_patience = 1 # кол-во эпох без улучшений
RLPOP_verbose = 1 # выводить ли прогресс изменения шага сходимости в его процессее
RLPOP_mode = 'auto' # выбирает, уменьшать шаг сходимости при росте величины или при её уменьшении
RLPOP_min_delta = 0.001 # порог изменения отслеживаемого значения
RLPOP_cooldown = 2 # количество эпох до возобновления работы после изменения шага сходимости
RLPOP_min_lr = 0 # минимальное значение шага сходимости

    # для CallbackList
MODEL_VER = '14'
MODEL_NUM = '03'
MC_path = f"networks/CNN/checkpoints/model_id=v{MODEL_VER}n{MODEL_NUM}" + "/check-{epoch:04d}.keras"

    # для CallbackList
CBL_add_history = True # вызывать ли колбэк History (если он не был довавлен вручную)
CBL_add_progbar = True # вызывать ли колбэк ProgbarLogger (если он не был довавлен вручную)
    
#///////////////////////////////// для тренировки

FIT_batch_size = 4 # размерpython concat lists bach при обучении/тестировании1
#FIT_shuffle = True # перемешивать ли данные
FIT_verbose = True # выводить ли прогресс обучения в его процессее
FIT_epochs = 5 #40 # количество эпох обучения
#FIT_validation_split = 0.10 #0.20 # процент валидационных данных, отсекаемых из тестовой выборки

In [ ]:
#model = mv.get_model_v10(CROP_SIZE)
'''DROP = 0.1

def bnorm(layer, drop_percent):
    #return Dropout(drop_percent)(BatchNormalization()(layer))
    return Dropout(drop_percent)(layer)
    #return BatchNormalization()(layer)'''

augment_data = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(1),
  layers.RandomTranslation(0.4,0.4,fill_mode="reflect"),
  layers.RandomZoom(0.2,0.2,fill_mode="reflect")
])
.to_excel
input_data = Input((16,16,64), name = 'input_data')

aug_data = augment_data(input_data)

# 12
dconv_1_1 = Conv2D(128, (3,3), dilation_rate=(2, 2), activation='relu', name='dconv_1_1')(aug_data)

# 6
dconv_1_2 = Conv2D(128, (3,3), dilation_rate=(5, 5), activation='relu', name='dconv_1_2')(aug_data)
up_1_2 = UpSampling2D(2, interpolation='bilinear', name='up_1_2') (dconv_1_2)

# 4
dconv_1_3 = Conv2D(128, (3,3), dilation_rate=(6, 6), activation='relu', name='dconv_1_3')(aug_data)
up_1_3 = UpSampling2D(3, interpolation='bilinear', name='up_1_3') (dconv_1_3)

# 2
dconv_1_4 = Conv2D(128, (3,3), dilation_rate=(7, 7), activation='relu', name='dconv_1_4')(aug_data)
up_1_4 = UpSampling2D(6, interpolation='bilinear', name='up_1_4') (dconv_1_4)


conc_1_1 = concatenate([dconv_1_1, up_1_2, up_1_3, up_1_4],axis=3, name='conc_1_1')


conv_2_1 = Conv2D(512, (3,3), dilation_rate=(2, 2), activation='relu', padding='same', name='conv_2_1')(conc_1_1)
conv_2_2 = Conv2D(512, (3,3), dilation_rate=(2, 2), activation='relu', padding='same', name='conv_2_2')(conv_2_1)
conv_2_3 = Conv2D(512, (3,3), dilation_rate=(2, 2), activation='relu', padding='same', name='conv_2_3')(conv_2_2)
pool_2_1 = MaxPooling2D((2,2), strides=2, name='pool_2_1')(conv_2_3)

conv_2_4 = Conv2D(1024, (2,2), activation='relu', name='conv_2_4')(pool_2_1)
conv_2_5 = Conv2D(1024, (2,2), activation='relu', name='conv_2_5')(conv_2_4)
conv_2_6 = Conv2D(1024, (2,2), activation='relu', name='conv_2_6')(conv_2_5)
conv_2_7 = Conv2D(1024, (2,2), activation='relu', name='conv_2_7')(conv_2_6)
pool_2_2 = MaxPooling2D((2,2), strides=2, name='pool_2_2')(conv_2_7)

d_4_1 = Dense(1024, activation='linear', name='d_4_1')(Flatten(name='flat_3_1')(pool_2_2))
d_4_2 = Dense(512, activation='linear', name='d_4_2')(d_4_1)
d_4_3 = Dense(256, activation='linear', name='d_4_3')(d_4_2)
d_4_4 = Dense(128, activation='linear', name='d_4_4')(d_4_3)
d_4_5 = Dense(64, activation='linear', name='d_4_5')(d_4_4)
d_4_6 = Dense(16, activation='linear', name='d_4_6')(d_4_5)
d_4_7 = Dense(4, activation='linear', name='d_4_7')(d_4_6)

output_def_bool = Dense(1, activation='sigmoid', name='output_def_bool')(d_4_7)
    
model = keras.Model([input_data], [output_def_bool], name='model')

#model = keras.models.load_model('networks/CNN/id=v13n01_in(16x16x64)_out(1)_train=0dot0_test=0dot0.keras')

model.compile(optimizer=CMP_solver, loss=CMP_loss_funcs, metrics=CMP_metrics)

print(model.summary())

In [ ]:
tf.keras.utils.plot_model(
    model,
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=200,
    show_layer_activations=False,
    show_trainable=False,
)

In [ ]:
# Создание и настройка колбэков

callback_list = [] # массив колбэков до подачи в колбек "callbacklist"

'''
callback_list.append(keras.callbacks.EarlyStopping(
            monitor = ES_monitor_parametr, 
            min_delta = ES_min_delta, 
            patience = ES_patience,
            restore_best_weights = ES_save_best_weights
            ))'''

callback_list.append(keras.callbacks.ModelCheckpoint(
    MC_path, 
    verbose=1, 
    save_weights_only=False,
    save_freq='epoch'))

callback_list.append(keras.callbacks.ReduceLROnPlateau(
            monitor = RLPOP_monitor_parametr, 
            factor = RLPOP_factor, 
            patience = RLPOP_patience, 
            verbose = RLPOP_verbose,
            mode = RLPOP_mode, 
            min_delta = RLPOP_min_delta, 
            cooldown = RLPOP_cooldown, 
            min_lr = RLPOP_min_lr
            ))

FIT_callback_list = keras.callbacks.CallbackList(
            callbacks = callback_list, 
            add_history = CBL_add_history, 
            add_progbar = CBL_add_progbar, 
            model = model
            )

In [ ]:
history = model.fit(dataset['train']['x'], dataset['train']['bin'],
                    batch_size = FIT_batch_size, 
                    epochs = FIT_epochs, 
                    verbose = FIT_verbose, 
                    shuffle=True,
                    validation_data = (dataset['val']['x'], dataset['val']['bin']), 
                    callbacks = FIT_callback_list)

In [ ]:
FONT_SIZE = 15
for key1,key2 in zip(['BinaryAccuracy','Precision','MeanSquaredError','TruePositives','TrueNegatives'],
                     ['loss','Recall','AUC','FalseNegatives','FalsePositives']):
    fig, axes = plt.subplots(1,2)

    fig.set_figwidth(22)
    fig.set_figheight(8)
    
    axes[0].plot(history.history[key1], 
             label='Train dataset',  linewidth=1.5, color='blue')
    axes[0].plot(history.history[f'val_{key1}'], linestyle = '--', 
             label='Validation dataset',  linewidth=3, color='red')
    axes[0].set_xlabel('Epoch number', fontsize=FONT_SIZE)
    axes[0].set_ylabel(f'{key1} value', fontsize=FONT_SIZE)
    axes[0].set_title(f"Learning process {key1} plot", fontsize=FONT_SIZE, pad=15)
    axes[0].tick_params(axis='both', which='both', labelsize = FONT_SIZE)
    axes[0].minorticks_on()
    axes[0].grid(which='major', linewidth=2)
    axes[0].grid(which='minor', color = 'gray', linestyle = ':')
    axes[0].legend(fontsize = FONT_SIZE, facecolor = "white", loc = 'upper right')

    axes[1].plot(history.history[key2], 
             label='Train dataset',  linewidth=1.5, color='blue')
    axes[1].plot(history.history[f'val_{key2}'], linestyle = '--', 
             label='Validation dataset',  linewidth=3, color='red')
    axes[1].set_xlabel('Epoch number', fontsize=FONT_SIZE)
    axes[1].set_ylabel(f'{key2} value', fontsize=FONT_SIZE)
    axes[1].set_title(f"Learning process {key2} plot", fontsize=FONT_SIZE, pad=15)
    axes[1].tick_params(axis='both', which='both', labelsize = FONT_SIZE)
    axes[1].minorticks_on()
    axes[1].grid(which='major', linewidth=2)
    axes[1].grid(which='minor', color = 'gray', linestyle = ':')
    axes[1].legend(fontsize = FONT_SIZE, facecolor = "white", loc = 'upper right')

    plt.show()

In [ ]:
'''FONT_SIZE = 15
for key in [k for k in history.history.keys() if not k.startswith('val')]:
    fig, ax = plt.subplots()

    fig.set_figwidth(12)
    fig.set_figheight(8)
    
    plt.plot(history.history[key], 
             label='Train dataset',  linewidth=1.5, color='blue')
    plt.plot(history.history[f'val_{key}'], linestyle = '--', 
             label='Validation dataset',  linewidth=3, color='red')
    
    ax.set_xlabel('Epoch number', fontsize=FONT_SIZE)
    ax.set_ylabel(f'{key} value', fontsize=FONT_SIZE)
    ax.set_title(f"Learning process {key} plot", fontsize=FONT_SIZE, pad=15)
    
    ax.patch.set_alpha(0)
    
    #  Устанавливаем форматирование делений:
    ax.tick_params(axis='both', which='both', labelsize = FONT_SIZE)
    
    # Вывод и настройка сетки
    ax.minorticks_on()
    ax.grid(which='major', linewidth=2)
    ax.grid(which='minor', color = 'gray', linestyle = ':')
    
    ax.legend(fontsize = FONT_SIZE, facecolor = "white", loc = 'upper right')
    
    plt.show()'''

In [ ]:
# сохранинеи процесса обучения в файл
learn_df = pd.DataFrame.from_dict(history.history)
learn_df.to_excel(f"networks/CNN/checkpoints/model_id=v{MODEL_VER}n{MODEL_NUM}/learning_df.xlsx")
# сохранение модели в файл
model.save(f"networks/CNN/id=v{MODEL_VER}n{MODEL_NUM}_in({CROP_SIZE}x{CROP_SIZE}x64)_out(1)_train=0dot0_test=0dot0.keras")

#### 